### Colorization of panchromatic space images with Generative Adversarial Network

Author: Patrycja Cieplicka

Date: 12 Jan 2020

Implementation of needed functions to load/save/analyze data

In [1]:
from ipynb.fs.full.packages import *

Using TensorFlow backend.


In [2]:
def load_color(path):

    #sort names of images
    filenames_color = []
    for filename in os.listdir(path+'/RGB-PanSharpen'):
        filenames_color.append(filename)
    filenames_color.sort()

    #load color images
    Y= []
    for filename in filenames_color:
        if filename == 'RGB-PanSharpen_AOI_3_Paris_img107.tif.aux.xml': continue
        with rasterio.open(path +'/RGB-PanSharpen/' + filename, 'r') as src:

            new = img = np.empty((src.width, src.height, src.count), dtype ='uint8')
            img = np.empty((src.width, src.height, src.count))
            img[:, :, 0] = src.read(1)   #channel red
            img[:, :, 1] = src.read(2)   #channel green
            img[:, :, 2] = src.read(3)   #channel blue
            
            #add contrast
            p2 = np.percentile(img[:,:,1], 2)
            p98 = np.percentile(img[:,:,1],98)
            img = (img - p2)/(p98 - p2)

            #convert to 8-bit (0-255)
            new = bytescale(img)
            new = resize(new,(128,128), anti_aliasing=True)

            Y.append(new)

    Y = np.array(Y, dtype=float)

    return Y

In [10]:
def load_gray(path):

    #sort names of images
    filenames_grey = []
    for filename in os.listdir(path+'/PAN'):
        filenames_grey.append(filename)
    filenames_grey.sort()

    #load grayscaled images
    X= []
    for filename in filenames_grey:
        if filename == '.ipynb_checkpoints' : continue
        with rasterio.open(path +'/PAN/' + filename, 'r') as src:
            pan = np.empty((src.width, src.height, src.count))
            pan[:, :, 0] = src.read(1) #panchromatic channel
            
            #add contrast
            p2 = np.percentile(pan[:, :, 0], 2)
            p98 = np.percentile(pan[:, :, 0],98)
            pan[:, :, 0] = (pan[:, :, 0] - p2)/(p98 - p2)

            #convert to 8-bit (0-255)
            new = bytescale(pan)
            new = resize(new,(128,128), anti_aliasing=True)
            
            X.append(new)
            
    X = np.array(X, dtype ='float')
    
    return X

In [17]:
def predict_image_ab(X_l, Xlab, model):
    #Input: X_l - test images to colorize (L channel normalized)
    #Xlab - empty, to save generated image in Lab
    #model - loaded model colorization
    #Output: col - generated images in RGB
    
    col = []
    output = model.predict(X_l)  #predtict ab channel(normalized)

    output = output * 128        #ab channels
    X_l = X_l * 100              #L channel
    
    #create colorized image from L and ab channels
    for j in range(len(output)):
            photo = np.zeros((128, 128, 3))
            photo[:,:,1:] = output[j]
            photo[:,:,0] = X_l[j][:,:,0]
            col.append(lab2rgb(photo))       #colorized image to RGB
            Xlab.append(photo)
            
    col = np.array(col, dtype=float)
    Xlab = np.array(Xlab, dtype=float)
    return col

In [12]:
def predict_image_lab(X_l, Xlab, model):
    #Input: X_l - test images to colorize (L channel normalized)
    #Xlab - empty, to save generated image in Lab
    #model - loaded model colorization
    #Output: col - generated images in RGB
    
    col = []
    output = model.predict(X_l)  #predtict lab channel(normalized)

    output[:,:,:,1:] = output[:,:,:,1:] * 128    #ab channels
    output[:,:,:,0] = output[:,:,:,0] * 100      #L channel
    
    #create colorized image from L and ab channels
    for j in range(len(output)):
            photo = np.zeros((128, 128, 3))
            photo = output[j]
            col.append(lab2rgb(photo))    #colorized image to RGB
            Xlab.append(photo)
            
    col = np.array(col, dtype=float)
    Xlab = np.array(Xlab, dtype=float)
    return col

In [13]:
def predict_image_rgb(X_l, model):
    #Input: X_l - test images to colorize (L channel normalized)
    #model - loaded model colorization
    #Output: col - generated images in RGB
    
    col = []
    output = model.predict(X_l)  #predict rgb channles (normalized) 
    
    col = np.array(output*255, dtype=float)
    return col

In [14]:
def meanSquarredError(Xgen, Ytrue):
    #Input: Xgen - generated images in RGB
    #Ytrue - real images in RGB
    #Output: error_mean - Mean Squarred Error
    
    n_images = 214
    summ = []
    summ_ch = np.zeros([1,n_images])
    for j in range(0,3):
        for i in range(0, n_images):
            summ.append( np.sum((255*Xgen[i,:,:,j] - 255*Ytrue[i,:,:,j]) **2) )
        summ_ch = np.add(np.array(summ, dtype='float'), summ_ch)
        summ = []
    
    error_mse = summ_ch / (128*128*3)
    error_mean = np.sum(error_mse)/n_images
    
    return error_mean

In [15]:
def PSNRatio(error_mean):
    #Input: error_mean - Mean Squarred Error
    #Output: PSNR_Value - Peak to Noise Ratio
    
    PSNR_Value = 10 * math.log10( 255**2 / error_mean)
    return PSNR_Value

In [18]:
def DeltaE(Xlab, Ylab):
    #Input: Xlab - generated images in Lab
    #Ylab - real images in Lab
    #Output: delE - Delta Eab
    
    n_images = 214
    sumdeltaE = 0
    sumimag = 0
    for image in range(0,n_images):
        for i in range (0, 128):
            for j in range (0, 128):
                deltaE = sqrt((Ylab[image, i, j ,0] - Xlab[image, i, j ,0])**2 + (Ylab[image, i, j ,1] - Xlab[image, i, j ,1])**2 + (Ylab[image, i, j ,2] - Xlab[image, i, j ,2])**2 )
                sumdeltaE = deltaE + sumdeltaE
        imag = sumdeltaE / (128*128)
        sumdeltaE = 0
        sumimag = imag + sumimag

    delE = sumimag / n_images
    
    return delE